In [1]:
import pandas as pd
import time
from requests import get
import re
import numpy as np

full_data = pd.read_pickle('data/match_data.pkl')

In [2]:
full_data['alliances.blue.score']

0      271
1      258
2      267
3      201
4      192
5      165
6      269
7      142
8      105
9      122
10     369
11     316
12      82
13     192
14     176
15     477
16      79
17     101
18     211
19     203
20     140
21     298
22     188
23     251
24     469
25     155
26     465
27     247
28     383
29     267
      ... 
77      55
78     371
79      92
80     264
81     263
82     353
83     174
84     233
85     458
86     351
87     184
88     254
89     305
90      59
91     267
92     109
93     367
94     472
95     332
96     429
97     412
98     371
99     446
100    446
101    226
102    160
103    333
104    189
105    290
106    336
Name: alliances.blue.score, Length: 14281, dtype: int64

In [3]:
full_data.columns

Index(['actual_time', 'alliances.blue.dq_team_keys', 'alliances.blue.score',
       'alliances.blue.surrogate_team_keys', 'alliances.blue.team_keys',
       'alliances.red.dq_team_keys', 'alliances.red.score',
       'alliances.red.surrogate_team_keys', 'alliances.red.team_keys',
       'comp_level', 'event_key', 'key', 'match_number', 'post_result_time',
       'predicted_time', 'score_breakdown',
       'score_breakdown.blue.adjustPoints',
       'score_breakdown.blue.autoOwnershipPoints',
       'score_breakdown.blue.autoPoints',
       'score_breakdown.blue.autoQuestRankingPoint',
       'score_breakdown.blue.autoRobot1', 'score_breakdown.blue.autoRobot2',
       'score_breakdown.blue.autoRobot3', 'score_breakdown.blue.autoRunPoints',
       'score_breakdown.blue.autoScaleOwnershipSec',
       'score_breakdown.blue.autoSwitchAtZero',
       'score_breakdown.blue.autoSwitchOwnershipSec',
       'score_breakdown.blue.endgamePoints',
       'score_breakdown.blue.endgameRobot1',
      

In [4]:
import tensorflow as tf

class tf_opr:
    def __init__(self, n_teams = 8000, average_value = 100):
        self.x_ = x_ = tf.placeholder(tf.int32, [None, 3])
        self.lookup_x = lookup_x = tf.placeholder(tf.int32, [None])
        self.y_ = y_ = tf.placeholder(tf.float32, [None, 1])
        self.embeddings = embeddings = tf.Variable(tf.constant(average_value, shape=[n_teams, 1],  dtype=tf.float32))
        self.lookup = lookup = tf.nn.embedding_lookup(embeddings, x_)
        self.xlookup = xlookup = tf.nn.embedding_lookup(embeddings, lookup_x)
        self.y = y = tf.reduce_sum(lookup, axis = 1)
        self.error = error = tf.reduce_mean(tf.square(y - y_))
        self.optimizer = optimizer = tf.train.AdamOptimizer(1).minimize(error)
        self.sess = sess = tf.Session()
        sess.run(tf.global_variables_initializer())
    def fit(self, x, y, iters):
        with self.sess.as_default() as sess:
            for i in range(iters):
                _, e = sess.run([self.optimizer, self.error],feed_dict={self.y_: y, self.x_ : x})
                if(i % 600 == 0):
                    print(e)
    def predict(self, x):
        with self.sess.as_default() as sess:
            y = sess.run(self.y, feed_dict = {self.x_ : x})
            return y
    def get_embeddings(self):
        with self.sess.as_default() as sess:
            emb = sess.run(self.embeddings)
            return emb
    def lookup_teams(self, teams):
        with self.sess.as_default() as sess:
            e = sess.run(self.xlookup, feed_dict={self.lookup_x : teams})
            return e
        
class tf_advanced:
    def __init__(self, n_teams = 8000, average_value = 0.5):
        self.x_ = x_ = tf.placeholder(tf.int32, [None, 3])
        self.lookup_x = lookup_x = tf.placeholder(tf.int32, [None])
        self.y_ = y_ = tf.placeholder(tf.float32, [None, 1])
        self.embeddings = embeddings = tf.Variable(tf.constant(average_value, shape=[n_teams, 1],  dtype=tf.float32))
        self.lookup = lookup = tf.nn.embedding_lookup(embeddings, x_)
        self.xlookup = xlookup = tf.nn.embedding_lookup(embeddings, lookup_x)
        self.y = y = tf.sigmoid(tf.reduce_sum(lookup, axis = 1))
        self.error = error = tf.reduce_mean(tf.square(y - y_))
        self.optimizer = optimizer = tf.train.AdamOptimizer(1).minimize(error)
        self.sess = sess = tf.Session()
        sess.run(tf.global_variables_initializer())
    def fit(self, x, y, iters):
        with self.sess.as_default() as sess:
            for i in range(iters):
                _, e = sess.run([self.optimizer, self.error],feed_dict={self.y_: y, self.x_ : x})
                if(i % 600 == 0):
                    print(e)
    def predict(self, x):
        with self.sess.as_default() as sess:
            y = sess.run(self.y, feed_dict = {self.x_ : x})
            return y
    def get_embeddings(self):
        with self.sess.as_default() as sess:
            emb = sess.run(self.embeddings)
            return emb
    def lookup_teams(self, teams):
        with self.sess.as_default() as sess:
            e = sess.run(self.xlookup, feed_dict={self.lookup_x : teams})
            return e

def frcstr_to_frcnum(base_df):
    base_df = base_df[np.isnan(base_df['score_breakdown.blue.totalPoints']) == False]
    base_df = base_df[base_df['alliances.blue.score'] > 0]
    for team in ['blue', 'red']:
        team_keys = 'alliances.{}.team_keys'.format(team)
        rep = []
        for keys in base_df[team_keys]:
            rep.append([int(k.replace('frc','')) for k in keys])
        base_df[team_keys] = rep
    return base_df
def frcnum_individualize(base_df):
    for team in ['blue', 'red']:
        keys = base_df['alliances.{}.team_keys'.format(team)].values
        for i in range(3):
            base_df[team+'team'+str(i+1)] = [k[i] for k in keys]
    return base_df    
def opr_data(base_df, opr_col = 'score_breakdown.{}.totalPoints'):
    x = []
    y = []
    for team in ['blue', 'red']:
        x.extend(base_df['alliances.{}.team_keys'.format(team)].values)
        y.extend(base_df[opr_col.format(team)].values)
    return x,y
def replacer(arr, rp):
    return [rp[a] for a in arr]

In [5]:
full_data = frcstr_to_frcnum(full_data)
full_data = frcnum_individualize(full_data)

In [6]:
team_str_int_dict = {'blue' : 0, 'red' : 1, '' : 2}
team_str_blue_dict = {'blue' : 1, 'red' : 0, '' : 0.5}
team_str_red_dict = {'blue' : 0, 'red' : 1, '' : 0.5}
full_data['red_win'] = replacer(full_data['winning_alliance'], team_str_red_dict)
full_data['blue_win'] = replacer(full_data['winning_alliance'], team_str_blue_dict)
full_data['winning_alliance'] = replacer(full_data['winning_alliance'], team_str_int_dict)
frcnum_individualize(full_data)

train_data = full_data[full_data['comp_level'] =='qm']
val_data = full_data[full_data['comp_level'] !='qm']

In [7]:
oprs = ['adjustPoints','autoPoints', 'autoOwnershipPoints', 'autoRunPoints', 'autoScaleOwnershipSec', 'autoScaleOwnershipSec',
'endgamePoints', 'foulPoints', 'teleopOwnershipPoints', 'teleopPoints', 
'teleopScaleOwnershipSec','teleopSwitchOwnershipSec', 'vaultPoints', 'vaultLevitateTotal', 'vaultForceTotal', 'vaultBoostTotal',
 'totalPoints']

team_data = pd.DataFrame()
team_data['team'] = pd.unique(full_data[['blueteam1','blueteam2','blueteam3', 'redteam1','redteam2','redteam3']].values.ravel('K'))
team_data = team_data.sort_values(['team'])

for opr in oprs:
    print()
    print(opr)
    tfm = tf_opr()
    x, y = opr_data(train_data, 'score_breakdown.{}.'+opr)
    tfm.fit(x, np.transpose([y]), 1201)
    for team in ['blue', 'red']:
        for i in range(3):
            team_data[opr] = tfm.lookup_teams(team_data['team'])
            train_data[team+'team'+str(i+1)+'_'+opr] = tfm.lookup_teams(train_data[team+'team'+str(i+1)])
            val_data[team+'team'+str(i+1)+'_'+opr] = tfm.lookup_teams(val_data[team+'team'+str(i+1)])


tfm = tf_advanced()
x, y = opr_data(full_data, '{}_win')
tfm.fit(x, np.transpose([y]), 1201)
for team in ['blue', 'red']:
    for i in range(3):
        team_data['winningness'] = tfm.lookup_teams(team_data['team'])
        train_data[team+'team'+str(i+1)+'_'+'winningness'] = tfm.lookup_teams(train_data[team+'team'+str(i+1)])  
        val_data[team+'team'+str(i+1)+'_'+'winningness'] = tfm.lookup_teams(val_data[team+'team'+str(i+1)])  


adjustPoints
90000.07
7.614809e-14
2.9850862e-19


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



autoPoints
76837.055
66.074005
66.07105

autoOwnershipPoints
84058.28
50.59958
50.595592

autoRunPoints
82496.01
7.290664
7.290342

autoScaleOwnershipSec
89628.74
2.520897
2.520681

autoScaleOwnershipSec
89628.734
2.520897
2.5206807

endgamePoints
66378.34
297.443
297.43842

foulPoints
79455.2
659.8419
659.8266

teleopOwnershipPoints
23950.246
3560.2803
3560.2803

teleopPoints
13769.233
4928.701
4928.701

teleopScaleOwnershipSec
61935.49
1569.0323
1569.0215

teleopSwitchOwnershipSec
39985.68
1148.058
1148.0574

vaultPoints
76264.484
109.23151
109.226524

vaultLevitateTotal
88582.5
0.89330137
0.8932356

vaultForceTotal
89346.37
0.9924371
0.9923501

vaultBoostTotal
89181.75
1.0389265
1.0388266

totalPoints
12354.555
6083.9346
6083.9355
0.35050213
0.16522393
0.16242443


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
team_data[team_data.team.isin(state_teams)].to_csv('state.csv', index = False)

In [9]:
pembroke_teams = [900, 2059, 2642, 2655, 2682, 3196, 3229, 3336, 3402, 3459, 3506, 3661, 3796, 3822, 4290, 4534, 4795, 4829, 4935, 5160, 5518, 5544, 5854, 6003, 6004, 6214, 6240, 6500, 6512,6565, 6729, 6894, 6899, 6908, 7029, 7265, 7270]
state_teams = [435, 587, 900, 1533, 2059, 2642, 2655, 2682, 3196, 3229, 3459, 3506, 3737, 4290, 4561, 4795, 5160, 5190, 5511, 5518, 6003, 6215, 6500, 6502, 6565, 6729, 6888, 6908, 6932]

In [10]:
forsyth_teams = [435, 900, 1225, 1533, 2640, 2655, 3215, 3402, 3506, 3680, 3737, 3845, 4290, 4534, 4767, 4795, 4816, 4829, 5160, 5544, 5607, 5679, 5727, 5762, 6215, 6426, 6502, 6565]


In [11]:
train_data.to_pickle('data/train_data.pkl')
val_data.to_pickle('data/val_data.pkl')
team_data.to_pickle('data/team_data.pkl')
#Expected ranks are 14th and 10th for Pembroke and Forsyth respectively

In [12]:
team_data.sort_values(['winningness'])

,team,adjustPoints,autoPoints,autoOwnershipPoints,autoRunPoints,autoScaleOwnershipSec,endgamePoints,foulPoints,teleopOwnershipPoints,teleopPoints,teleopScaleOwnershipSec,teleopSwitchOwnershipSec,vaultPoints,vaultLevitateTotal,vaultForceTotal,vaultBoostTotal,totalPoints,winningness
825,5605,-8.718581e-10,4.991555,2.534981,2.456352,0.144538,2.654175,5.489935,-9.135564,-6.005771,-4.704297,-1.460306,0.475702,0.601532,-0.025219,-0.481177,4.490024,-135.678360
3205,7292,-1.565765e-10,-0.776045,-0.398571,-0.377453,0.007059,4.134604,-2.966822,-19.074713,-15.482327,-14.168111,-2.522161,-0.542181,0.438101,-0.092608,-0.453928,-19.226831,-118.495277
982,2477,-6.303550e-11,6.043288,1.838603,4.204697,0.224993,8.375892,-5.949353,27.644728,41.330032,0.660218,28.688105,5.309372,0.713419,0.157569,0.190881,41.439396,-100.905907
3155,5308,1.115609e-09,0.092255,-2.225868,2.318168,-0.099085,0.400016,-6.401443,-0.365937,1.805559,-6.494659,7.294176,1.771635,0.146366,0.276377,-0.068400,-4.506519,-94.950012
1023,4329,-2.183213e-10,1.784493,-2.103192,3.887730,0.088441,11.422370,1.959357,56.335716,89.628014,11.134897,38.045540,21.869904,1.212193,1.581674,1.580085,93.368050,-93.568748
1695,5559,5.682186e-12,4.582053,0.728112,3.853940,-0.217256,8.943083,-1.336838,13.763014,29.485373,-0.933747,15.894750,6.779295,0.874425,0.530397,-0.048962,32.739849,-92.139809
1841,5448,1.020008e-11,7.022371,2.304143,4.718234,0.355178,16.336336,-2.391498,15.099062,37.707222,-12.430970,26.978500,6.271902,0.912444,0.160329,0.181606,42.342396,-90.632271
212,996,1.146362e-10,2.699627,-1.831907,4.531514,-0.820591,6.332451,10.261898,18.726017,28.381903,-7.647189,28.085592,3.323431,1.075393,-0.010108,-0.400587,41.352272,-90.094810
1764,5069,-2.859818e-12,6.473555,1.967112,4.506441,0.001394,13.658978,2.754083,35.503510,57.670673,14.483850,21.201714,8.508212,0.972450,0.384597,0.344596,66.909851,-89.991776
3178,3984,4.509911e-10,2.180482,-1.930478,4.111003,-0.178028,11.255351,-5.147668,2.210180,14.718981,-6.662097,11.276103,1.253423,0.776617,-0.063141,-0.462777,11.767422,-87.187431


In [13]:
team_data[team_data['team'] in state_teams]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().